# Sentiment Analysis for Twitter

## Overview

This tutorial is going to introduce some simple tools for detecting sentiment in Tweets. We will be using a set of tools called the Natural Language Toolkit ([NLTK](http://www.nltk.org)). This is collection of software written in the [Python](https://www.python.org) programming language. An important design goal behind Python is that it should be easy to read and fun to use, so well-suited for beginners. A similar motivation inspired NLTK: it should make complex tasks easy to carry out, and it should be written in a way that would allow users to inspect and understand the code.

Why is this relevant? Well, a lot of software these days is built to be easy to use, but hard to inspect. For example, smartphones have a lot of slick apps on them, but very few people have the expertise to look under the hood to find out how they work. NLTK has quite the opposite approach: you are actively encouraged to discover how the code works. However, your level of understanding will depend heavily on how far you get to grips with Python itself.

This tutorial is written using the [Ipython](http://ipython.org) framework. This allows text to be interspersed by fragments of code, occuring in special "cells". Just below is a cell where we are using Python to do a simple calculation:

In [1]:
3 + 4

7

Some of the cells will contain snippets of code that are necessary for the big story to work, but which you don't need to understand. We'll try to make it clear when it's important for you to pay attention to one of the cells. 

## Twitter

As you know, people are tweeting all the time. The rate varies, with about 6,000 per second being the average, but when I last checked, the [rate was over 10,000 Tweets per second](http://www.internetlivestats.com/one-second/#tweets-band). So, a lot. Twitter kindly allows people to tap into a small sample of this stream &mdash; unless you're able to pay, the sample is at most 1% of the total stream. 

Here's a tiny snapshow to Tweets, taken at 13.00 pm on Sunday 11 November 2015. By using the keywords `'love, hate'`, we restrict our sample to just those Tweets containing one or both of those words.

In [3]:
import nltk # load up the NLTK library
from nltk.twitter import Twitter
tw = Twitter() # start a new client that connects to Twitter
tw.tweets(keywords='love, hate', limit=25) #filter Tweets from the public stream

Houses are like people - some you like and some you don't like - and once in a while there is one you love.
RT @halsey: @harmlesshalsey I don't consider any of this hate I'm clearing up misunderstandings be people are constantly reaching for reaso…
RT @MemeMinawya: love story ended with a pain💔
Listen to know what happened👇
@Mina_Atta_ 
https://t.co/ohwqaOiQKi … … … … … … … …

#DemForu…
RT @superlativehes: @Harry_Styles 
❝You're a charming gardener that makes my soul blossom❞
Thanks for making me happy ❃
Mind following me H…
RT @baaari_dr: Thank you for the love daw 😉 https://t.co/uIs4pT9JUU
i reallt hate slow updates!
RT @PopWrapped: @justinbieber Well smell a HUGE record breaker. Congrats on your best project to date, Justin. Lots of love. #Purpose
RT @K__amin: "I love you"

"ILYTAF"

"Apa tu ?"

"I love you too as friend"

Perbualan tamat disitu,burung-burung berkicauan,angin bertiupa…
RT @piedwagtail91: @JanSmithNL 
Hi Jan. Yes it's going ok. Hope yours is too. Love and light ☺
RT 

## Using a Twitter corpus

You too can sample Tweets in this way, but you'll need to set up your Twitter API keys according to [these instructions](http://www.nltk.org/howto/twitter.html). Since this is a bit of hassle, for the rest of this tutorial, we'll focus our attention on a sample of 20,000 Tweets that were collected at the end of April 2015. In order focus on Tweets about the UK general election, the public stream was filtered with the following set of terms:
```
david cameron, miliband, milliband, sturgeon, clegg, farage, tory, tories, ukip, snp, libdem
```
The following code cell allows us to get hold of this collection, and prints out the text of the first 15. You don't need to worry about the details of how this happens.

In [9]:
from nltk.corpus import twitter_samples
strings = twitter_samples.strings('tweets.20150430-223406.json')
for string in strings[:20]:
    print(string)

RT @KirkKus: Indirect cost of the UK being in the EU is estimated to be costing Britain £170 billion per year! #BetterOffOut #UKIP
VIDEO: Sturgeon on post-election deals http://t.co/BTJwrpbmOY
RT @LabourEoin: The economy was growing 3 times faster on the day David Cameron became Prime Minister than it is today.. #BBCqt http://t.co…
RT @GregLauder: the UKIP east lothian candidate looks about 16 and still has an msn addy http://t.co/7eIU0c5Fm1
RT @thesundaypeople: UKIP's housing spokesman rakes in £800k in housing benefit from migrants.  http://t.co/GVwb9Rcb4w http://t.co/c1AZxcLh…
RT @Nigel_Farage: Make sure you tune in to #AskNigelFarage tonight on BBC 1 at 22:50! #UKIP http://t.co/ogHSc2Rsr2
RT @joannetallis: Ed Milliband is an embarrassment. Would you want him representing the UK?!  #bbcqt vote @Conservatives
RT @abstex: The FT is backing the Tories. On an unrelated note, here's a photo of FT leader writer Jonathan Ford (next to Boris) http://t.c…
RT @NivenJ1: “@George_Osborne: Ed Mi

## Sentiment Analysis

It's clear that many of the people responsible for the Tweets shown above have strong opinions. We will use the term `sentiment analysis` to refer to the process of using software to figure out what these opinions actually are.

Although sentiment analysis is designed to work with written text, the way in which people express their feelings is often goes far beyond what they literally say. In spoken language, intonation will be important. And of course we often express emotion using no words at all, as illustrated in this picture from Darwin's book *The Expression of the Emotions*.

<a title="By Charles Darwin (author of volume); unknown photographer of plate [Public domain], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File%3APlate_depicting_emotions_of_grief_from_Charles_Darwin's_book_The_Expression_of_the_Emotions.jpg"><img align="left" width="512" alt="Plate depicting emotions of grief from Charles Darwin&'s book The Expression of the Emotions" src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/0c/Plate_depicting_emotions_of_grief_from_Charles_Darwin%27s_book_The_Expression_of_the_Emotions.jpg/512px-Plate_depicting_emotions_of_grief_from_Charles_Darwin%27s_book_The_Expression_of_the_Emotions.jpg"/></a>


In [6]:
input_file = twitter_samples.abspath("tweets.20150430-223406.json")

In [8]:
from nltk.twitter.util import json2csv
with open(input_file) as fp:
    json2csv(fp, 'tweets_text.csv', ['text'])

In [12]:
import pandas as pd
tweets = pd.read_csv('tweets_text.csv', header=0, encoding="utf8")
tweets.head(20)

,text
0,RT @KirkKus: Indirect cost of the UK being in ...
1,VIDEO: Sturgeon on post-election deals http://...
2,RT @LabourEoin: The economy was growing 3 time...
3,RT @GregLauder: the UKIP east lothian candidat...
4,RT @thesundaypeople: UKIP's housing spokesman ...
5,RT @Nigel_Farage: Make sure you tune in to #As...
6,RT @joannetallis: Ed Milliband is an embarrass...
7,RT @abstex: The FT is backing the Tories. On a...
8,RT @NivenJ1: “@George_Osborne: Ed Miliband pro...
9,LOLZ to Trickle Down Wealth. It's never trickl...


In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer